# Problem Statement

We would like to add a feature for recommended tickers and weights. 


## High Level Design

* This can be run weekly/fortnightly. 

* We will download data for the top 50 tickers for a given index. 

* Get the sharpe ratios based on criteria - 5 years data, 2 years data, 1 year data. Average them out and take the top 20, discard the rest

* Use the top 20, run Efficient Frontier for min volatility and max sharpe. Display these data with some text explanation (**ChatGPT??**)


In [29]:
import pandas as pd
import json 

from UtilFuncs.get_yfinance_data import *

### Step1: Download all data  

In [30]:
index_to_run = "^AXJO"

In [31]:

# Get all tickers for this index
with open('Resources/symbols_list.json', 'r') as json_file:
    symbols_dict = json.load(json_file)

tickers_names = symbols_dict[index_to_run]
tickers_all = [ticker.split(":")[0] for ticker in tickers_names]
# tickers_all

In [52]:
df_prices_all = pd.DataFrame()
# We'll run in batches of 5 to support multithreading
for i in range(10):
    start_index = 5 * i 
    end_index = start_index + 5
    tickers = tickers_all[start_index:end_index]
    print(tickers)

    if len(df_prices_all) == 0: 

        df_prices_all = get_securities_prices(tickers=tickers, download_tnx=False)

    else: 

        df_prices_all = pd.concat([df_prices_all, get_securities_prices(tickers=tickers, download_tnx=False)], axis="columns") 



['CBA.AX', 'BHP.AX', 'CSL.AX', 'WBC.AX', 'ANZ.AX']
[*********************100%%**********************]  5 of 5 completed
['FMG.AX', 'NAB.AX', 'MQG.AX', 'GMG.AX', 'WOW.AX']
[*********************100%%**********************]  5 of 5 completed
['WES.AX', 'TLS.AX', 'RIO.AX', 'WDS.AX', 'TCL.AX']
[*********************100%%**********************]  5 of 5 completed
['SQ2.AX', 'ALL.AX', 'COL.AX', 'SCG.AX', 'S32.AX']
[*********************100%%**********************]  5 of 5 completed
['NCM.AX', 'SUN.AX', 'QBE.AX', 'BXB.AX', 'FPH.AX']
[*********************100%%**********************]  5 of 5 completed
['COH.AX', 'ASX.AX', 'STO.AX', 'RHC.AX', 'AMC.AX']
[*********************100%%**********************]  5 of 5 completed
['ORG.AX', 'IAG.AX', 'SHL.AX', 'DXS.AX', 'BSL.AX']
[*********************100%%**********************]  5 of 5 completed
['APA.AX', 'TWE.AX', 'REA.AX', 'AIA.AX', 'DMP.AX']
[*********************100%%**********************]  5 of 5 completed
['CPU.AX', 'TAH.AX', 'VCX.AX', 'MGR.AX',

In [53]:
print(df_prices_all.shape)

(2529, 50)


In [55]:
df_prices_all.to_csv(f"Resources/Recommendations_{index_to_run}_stocks_data.csv")